<a href="https://colab.research.google.com/github/adildhkh/deep-learning/blob/main/object_detection_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install protobuf==3.20.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


In [1]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=.


In [2]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'


In [7]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), 'models/research'))
sys.path.append(os.path.join(os.getcwd(), 'models/research/slim'))


In [4]:
!pip install -U pip
!pip install tensorflow
!pip install tensorflow-addons
!pip install Cython
!pip install contextlib2
!pip install pillow
!pip install lxml
!pip install jupyter
!pip install matplotlib
!pip install pandas
!pip install tf-slim
!pip install pycocotools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling

In [8]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils


In [9]:
# Clone TensorFlow models repository
!git clone http://github.com/tensorflow/models.git

# Install protobuf and compile .proto files
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

# Install the Object Detection API
!cp object_detection/packages/tf2/setup.py . && python -m pip install .

fatal: destination path 'models' already exists and is not an empty directory.
cp: cannot stat 'object_detection/packages/tf2/setup.py': No such file or directory


In [11]:
import tensorflow as tf
# Configure GPU settings
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Placeholder to load videos from Google Drive (for you to replace)
INPUT_VIDEOS = ['catdog.mp4', 'safari.mp4']

# Download the SSD ResNet101 model from TensorFlow Model Zoo
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'
PATH_TO_MODEL_DIR = tf.keras.utils.get_file(fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8', origin=url, untar=True)

# Download the label map
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'
PATH_TO_LABELS = tf.keras.utils.get_file(fname='mscoco_label_map.pbtxt', origin=url, untar=False)

386527459/386527459 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5056/5056 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
import time
# Display first few lines of the label map
!head {PATH_TO_LABELS}

# Load the detection model
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print('Loading model...', end=' ')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}
Loading model... Done! Took 74.6931004524231 seconds


In [14]:
# Load label map data (for class visualization)
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Helper function to perform object detection on a single image
def detect_objects_in_image(image_np):
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=0.30,
        agnostic_mode=False
    )
    return image_np_with_detections

# Function to perform object detection on videos
def detect_objects_in_video(input_video):
    print(f'Running inference for {input_video}...', end=' ')
    video_reader = imageio.get_reader(input_video)
    video_writer = imageio.get_writer(f'{input_video.split(".")[0]}_annotated.mp4', fps=10)

    n_frames = 0
    t0 = time.time()
    for frame in video_reader:
        n_frames += 1
        new_frame = detect_objects_in_image(frame)
        video_writer.append_data(new_frame)
    fps = n_frames / (time.time() - t0)
    print(f"Frames processed: {n_frames}, speed: {fps:.2f} fps")
    video_writer.close()

In [16]:
!pip install imageio


In [19]:
import imageio
import numpy as np

In [22]:
# Run object detection on the videos
detect_objects_in_video(INPUT_VIDEOS[0])


Running inference for catdog.mp4... Frames processed: 311, speed: 0.69 fps


In [ ]:
detect_objects_in_video(INPUT_VIDEOS[1])